In [36]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [37]:
using JuMP, GLPK
using DataFrames
using CSV

In [38]:
#load cost data
con_generation = CSV.File("../data/conventional_generators.csv") |> DataFrame

#load wind_technicaldata
wind_generation = CSV.File("../data/wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame

# batteries
battery = CSV.File("../data/Battery.csv") |> DataFrame

# transmission_lines
transmission_lines = CSV.File("../data/transmission_lines.csv") |> DataFrame


Row,Transmission lines: From node,To node,Susceptance [per-unit],Capacity [MW]
,Int64,Int64,String7,Int64
1,1,2,0.0146,175
2,1,3,0.2253,175
3,1,5,0.0907,350
4,2,4,0.1356,175
5,2,6,0.205,175
6,3,9,0.1271,175
7,3,24,0.084,400
8,4,9,0.111,175
9,5,10,0.094,350


In [39]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
G = size(con_generation, 1)

# number of demand
D = size(demand_bids, 1)

# number of wind generators
W = size(wind_generation, 1)

# number of batteries
B = size(battery, 1)

# number of transmission lines
L = size(transmission_lines, 1)

# number of nodes
N = 24


24

In [40]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x_con$i")] = Float64[]
end

# For w variables, manually add each column. This is a one-time setup.
for i in 1:W
    result_df[!, Symbol("x_wind$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y$i")] = Float64[]
end

# For b variables, manually add each column. This is a one-time setup.
for i in 1:B
    result_df[!, Symbol("x_bat$i")] = Float64[]
end

# For l variables, manually add each column. This is a one-time setup.
for i in 1:L
    result_df[!, Symbol("x_line$i")] = Float64[]
end



equilibrium_df = DataFrame(hour = Int[])

# for each node a market price column
for i in 1:N
    equilibrium_df[!, Symbol("market_price_node$i")] = Float64[]
end



In [41]:
println(result_df)

0×73 DataFrame
 Row │ hour   x_con1   x_con2   x_con3   x_con4   x_con5   x_con6   x_con7   x_con8   x_con9   x_con10  x_con11  x_con12  x_wind1  x_wind2  x_wind3  x_wind4  y1       y2       y3       y4       y5       y6       y7       y8       y9       y10      y11      y12      y13      y14      y15      y16      y17      x_bat1   x_bat2   x_bat3   x_bat4   x_bat5   x_line1  x_line2  x_line3  x_line4  x_line5  x_line6  x_line7  x_line8  x_line9  x_line10  x_line11  x_line12  x_line13  x_line14  x_line15  x_line16  x_line17  x_line18  x_line19  x_line20  x_line21  x_line22  x_line23  x_line24  x_line25  x_line26  x_line27  x_line28  x_line29  x_line30  x_line31  x_line32  x_line33  x_line34 
     │ Int64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float

In [42]:
# Load demand bids data
demand_bids_all = [CSV.File("../data/demand_bids_hour/demand_hour_$(i-1).csv") |> DataFrame for i in 1:24]

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for every generator and hour
@variable(model, x_con[1:G, 1:24] >= 0)  # Power output variable for conventional generators
@variable(model, x_wind[1:W, 1:24] >= 0)  # Power output variable for wind generators
@variable(model, y[1:D, 1:24] >= 0)  # Demand variable
@variable(model, bat[1:B, 1:24])  # Battery variable
@variable(model, l[1:L, 1:24])  # Transmission line variable


# Add constraints for each plant
for g in 1:G
    for hour in 1:24
        @constraint(model, x_con[g, hour] <= con_generation[g, 6])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each wind farm
for w in 1:W
    for hour in 1:24
        @constraint(model, x_wind[w, hour] <= wind_profile[hour, w+1])  # Set the upper bound
        @constraint(model, x_wind[w, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each demand bid
for d in 1:D
    for hour in 1:24
        @constraint(model, y[d, hour] <= demand_bids_all[hour][d, 2])  # Set the upper bound
        @constraint(model, y[d, hour] >= 0)  # Set the lower bound
    end
end

# Add temporary for each generator
for g in 1:G
    for hour in 2:24
        @constraint(model, x_con[g, hour] <= x_con[g, hour - 1] + con_generation[g, 7])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= x_con[g, hour - 1] - con_generation[g, 8])  # Set the lower bound
    end
end

# add constraints for battery charging and discharging
for b in 1:B
    for hour in 1:24
        @constraint(model, bat[b, hour] <= battery[b, 5])  # Set the upper bound
        @constraint(model, bat[b, hour] >= -1 *battery[b, 5])  # Set the lower bound
    end
end

# add constraints for battery capacity
# sum all previous battery charging and discharging, they need to be below the battery capacity
for b in 1:B
    for hour in 2:24
        @constraint(model, sum(bat[b, h] for h in 1:hour) <= battery[b, 4])  # Set the upper bound
        @constraint(model, sum(bat[b, h] for h in 1:hour) >= 0)  # Set the lower bound
    end
end

# add constraints for transmission lines
for line in 1:L
    for hour in 1:24
        @constraint(model, l[line, hour] <= transmission_lines[line, 4])  # Set the upper bound
        @constraint(model, l[line, hour] >= -1 * transmission_lines[line, 4])  # Set the lower bound
    end
end

balance = Vector{Any}(undef, N*24)

# add balance constraint for each node and each demand_hour_
for hour in 1:24
    for node in 1:N
        balance[(hour-1)*24+node] = @constraint(model, sum(x_con[g, hour] for g in 1:G if con_generation[g, 2] == node) +
                            sum(x_wind[w, hour] for w in 1:W if wind_generation[w, 2] == node) +
                            sum(l[line, hour] for line in 1:L if transmission_lines[line, 1] == node) ==
                            sum(y[d, hour] for d in 1:D if demand_bids_all[hour][d, 1] == node))
        #set_name(c, "balance_constraint_node$(node)_hour$(hour)")
    end
end

# Define the objective function
@objective(model, Max, sum(demand_bids_all[hour][d, 3] * y[d, hour] for hour in 1:24, d in 1:D) -
                     sum(con_generation[g, 3] * x_con[g, hour] for hour in 1:24, g in 1:G))

#- sum(dual(balance[hour]) * bat[b, hour] for hour in 1:24, b in 1:B)



# Solve the model
optimize!(model)


# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")

    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))


else
    println("No optimal solution found")
end


Optimal solution found
Objective value: 1.0244594345210121e6


In [43]:
#save the duals of the balance constraints to the equilibrium_df
for hour in 1:24
    hour_results = []

    for node in 1:N
        push!(hour_results, dual(balance[(hour-1)*24+node]))
    end
    push!(equilibrium_df, [hour; hour_results...])
end



#save results to dataframe
CSV.write("results/optimization_results_marketprices.csv", equilibrium_df)



"results/optimization_results_marketprices.csv"

In [44]:
equilibrium_df

Row,hour,market_price_node1,market_price_node2,market_price_node3,market_price_node4,market_price_node5,market_price_node6,market_price_node7,market_price_node8,market_price_node9,market_price_node10,market_price_node11,market_price_node12,market_price_node13,market_price_node14,market_price_node15,market_price_node16,market_price_node17,market_price_node18,market_price_node19,market_price_node20,market_price_node21,market_price_node22,market_price_node23,market_price_node24
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0


In [45]:
#plot equilibrium_df market_price in a line plot
using Plots
plot(equilibrium_df.hour, equilibrium_df.market_price, label="Market Price", xlabel="Hour", ylabel="Price", title="Market Price by Hour", lw=2)

ArgumentError: ArgumentError: column name :market_price not found in the data frame

In [46]:
#plot equilibrium_df market_quantity in a bar chart
bar(equilibrium_df.hour, equilibrium_df.market_quantity, label="Market Quantity", xlabel="Hour", ylabel="Quantity", title="Market Quantity by Hour")

ArgumentError: ArgumentError: column name :market_quantity not found in the data frame

In [47]:
#save result_df to csv
CSV.write("results/market_clearing_model2.csv", result_df)

"results/market_clearing_model2.csv"